1. Read the Bernoulli Mixture Model Derivation.
2. Read about Stochastic Expectation-Maximization (EM) Algorithm: https://www.sciencedirect.com/science/article/pii/S0167947320302504.
3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.
4. Use the data from the paper: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
5. Perform categorical clustering using the Bernoulli Mixture Model with Stochastic EM Algorithm.
6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.
7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
8. Write the report in Latex, push to your github with the codes.

In [2]:
from ucimlrepo import fetch_ucirepo 
soybean = fetch_ucirepo(id=91) 
zoo = fetch_ucirepo(id=111) 
heart_disease = fetch_ucirepo(id=45) 
dermatology = fetch_ucirepo(id=33)
breast_cancer = fetch_ucirepo(id=15)
mushroom = fetch_ucirepo(id=73) 

In [3]:
import pandas as pd
X = soybean.data.features
y = soybean.data.targets 
soybean_df = pd.merge(X, y, left_index=True, right_index=True)

X = zoo.data.features
y = zoo.data.targets 
zoo_df = pd.merge(X, y, left_index=True, right_index=True)

X = heart_disease.data.features
y = heart_disease.data.targets 
heart_disease_df = pd.merge(X, y, left_index=True, right_index=True)

X = dermatology.data.features
y = dermatology.data.targets 
dermatology_df = pd.merge(X, y, left_index=True, right_index=True)

X = breast_cancer.data.features
y = breast_cancer.data.targets 
breast_cancer_df = pd.merge(X, y, left_index=True, right_index=True)

X = mushroom.data.features
y = mushroom.data.targets 
mushroom_df = pd.merge(X, y, left_index=True, right_index=True)


soybean_df = soybean_df.dropna()
zoo_df = zoo_df.dropna()
heart_disease_df = heart_disease_df.dropna()
dermatology_df = dermatology_df.dropna()
breast_cancer_df = breast_cancer_df.dropna()
mushroom_df = mushroom_df.dropna()

In [1]:
from scipy.special import logsumexp

class BernoulliMixture:
    
    def __init__(self, n_components, max_iter, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
    
    def fit(self,x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            # E-Step
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            # M-Step
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            # Compute new log_likelihood:
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        
    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()
    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        mu_place = np.where(np.max(mu, axis=0) <= 1e-15, 1e-15, mu)
        return np.tensordot(x, np.log(mu_place), (1,1))
        
    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self):
        self.mu = np.einsum('ik,id -> kd', self.gamma, self.x) / self.Neff[:,None] 
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

In [2]:
class StochasticBernoulliMixture(BernoulliMixture):
    
    def __init__(self, n_components, max_iter, tol=1e-3, batch_size=100):
        super().__init__(n_components, max_iter, tol)
        self.batch_size = batch_size
        
    def fit(self,x):
        self.x = x
        self.init_params()
        self.old_logL = self.score(x)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            # Shuffle data
            np.random.shuffle(self.x)
            for i in range(0, len(self.x), self.batch_size):
                x_batch = self.x[i:i+self.batch_size]
                # E-Step
                self.gamma = self.get_responsibilities(self.get_log_bernoullis(x_batch))
                self.remember_params()
                # M-Step
                self.get_Neff()
                self.get_mu()
                self.get_pi()
            # Compute new log_likelihood:
            self.logL = self.score(x)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

In [9]:
# Example usage
# Assuming you have your data stored in X, where each row represents a sample and each column represents a binary feature.

# Initialize the model
n_clusters = 3  # Number of clusters
model = BernoulliModel(n_clusters)

# Fit the model to the data
model.fit(zoo_df)

# Predict cluster labels for the data
labels = model.predict(zoo_df)

C:\Users\raymo\AppData\Local\Temp\ipykernel_11048\4233135795.py:18: RuntimeWarning: invalid value encountered in log
  log_likelihood[:, k] = np.sum(X * np.log(self.cluster_centers[k]) + (1 - X) * np.log(1 - self.cluster_centers[k]), axis=1)
C:\Users\raymo\AppData\Local\Temp\ipykernel_11048\4233135795.py:18: RuntimeWarning: invalid value encountered in log
  log_likelihood[:, k] = np.sum(X * np.log(self.cluster_centers[k]) + (1 - X) * np.log(1 - self.cluster_centers[k]), axis=1)
C:\Users\raymo\AppData\Local\Temp\ipykernel_11048\4233135795.py:18: RuntimeWarning: invalid value encountered in log
  log_likelihood[:, k] = np.sum(X * np.log(self.cluster_centers[k]) + (1 - X) * np.log(1 - self.cluster_centers[k]), axis=1)
C:\Users\raymo\AppData\Local\Temp\ipykernel_11048\4233135795.py:18: RuntimeWarning: invalid value encountered in log
  log_likelihood[:, k] = np.sum(X * np.log(self.cluster_centers[k]) + (1 - X) * np.log(1 - self.cluster_centers[k]), axis=1)
C:\Users\raymo\AppData\Local\Tem